In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/Users/muggallarahul/Documents/real_estate_project/data/crime_dataset_india.csv')

data

,Report Number,Date Reported,Date of Occurrence,Time of Occurrence,City,Crime Code,Crime Description,Victim Age,Victim Gender,Weapon Used,Crime Domain,Police Deployed,Case Closed,Date Case Closed
0,1,02-01-2020 00:00,01-01-2020 00:00,01-01-2020 01:11,Ahmedabad,576,IDENTITY THEFT,16,M,Blunt Object,Violent Crime,13,No,NaN
1,2,01-01-2020 19:00,01-01-2020 01:00,01-01-2020 06:26,Chennai,128,HOMICIDE,37,M,Poison,Other Crime,9,No,NaN
2,3,02-01-2020 05:00,01-01-2020 02:00,01-01-2020 14:30,Ludhiana,271,KIDNAPPING,48,F,Blunt Object,Other Crime,15,No,NaN
3,4,01-01-2020 05:00,01-01-2020 03:00,01-01-2020 14:46,Pune,170,BURGLARY,49,F,Firearm,Other Crime,1,Yes,29-04-2020 05:00
4,5,01-01-2020 21:00,01-01-2020 04:00,01-01-2020 16:51,Pune,421,VANDALISM,30,F,Other,Other Crime,18,Yes,08-01-2020 21:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40155,40156,01-08-2024 16:00,07-31-2024 03:00,31-07-2024 05:05,Kolkata,312,COUNTERFEITING,78,F,Firearm,Other Crime,12,No,NaN
40156,40157,31-07-2024 14:00,07-31-2024 04:00,31-07-2024 04:14,Mumbai,300,ILLEGAL POSSESSION,75,F,NaN,Other Crime,2,Yes,29-09-2024 14:00
40157,40158,02-08-2024 03:00,07-31-2024 05:00,31-07-2024 21:33,Kanpur,423,VEHICLE - STOLEN,68,M,Other,Violent Crime,6,No,NaN
40158,40159,01-08-2024 19:00,07-31-2024 06:00,31-07-2024 11:05,Patna,311,SEXUAL ASSAULT,11,M,Blunt Object,Violent Crime,1,No,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40160 entries, 0 to 40159
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Report Number       40160 non-null  int64 
 1   Date Reported       40160 non-null  object
 2   Date of Occurrence  40160 non-null  object
 3   Time of Occurrence  40160 non-null  object
 4   City                40160 non-null  object
 5   Crime Code          40160 non-null  int64 
 6   Crime Description   40160 non-null  object
 7   Victim Age          40160 non-null  int64 
 8   Victim Gender       40160 non-null  object
 9   Weapon Used         34370 non-null  object
 10  Crime Domain        40160 non-null  object
 11  Police Deployed     40160 non-null  int64 
 12  Case Closed         40160 non-null  object
 13  Date Case Closed    20062 non-null  object
dtypes: int64(4), object(10)
memory usage: 4.3+ MB


In [4]:
date_cols = ['Date Reported', 'Date of Occurrence', 'Date Case Closed']
for col in date_cols:
    data[col] = pd.to_datetime(data[col], errors='coerce')

/var/folders/h6/5b3yntr53k73ngkclllbcq_40000gn/T/ipykernel_1897/3830244867.py:3: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data[col] = pd.to_datetime(data[col], errors='coerce')


In [5]:
data['Weapon Used'].fillna('Unknown', inplace=True)
data['Date Case Closed'].fillna('Not Closed', inplace=True)

/var/folders/h6/5b3yntr53k73ngkclllbcq_40000gn/T/ipykernel_1897/586852524.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Not Closed' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  data['Date Case Closed'].fillna('Not Closed', inplace=True)


In [6]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['Victim Gender', 'Crime Description', 'Crime Domain', 'Weapon Used', 'Case Closed']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [7]:
import mysql.connector

db_config = {
    'user': 'root',
    'password': 'Rahul2002',  # Replace with actual credentials
    'host': 'localhost',
    'database': 'crime_analysis'
}

# Connect to MySQL and create database/table
conn = mysql.connector.connect(user=db_config['user'], password=db_config['password'], host=db_config['host'])
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS crime_analysis;")
conn.close()

In [8]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS crimes (
    ReportNumber INT,
    DateReported DATE,
    DateOccurrence DATE,
    TimeOccurrence TIME,
    City VARCHAR(255),
    CrimeCode INT,
    CrimeDescription INT,
    VictimAge INT,
    VictimGender INT,
    WeaponUsed INT,
    CrimeDomain INT,
    PoliceDeployed INT,
    CaseClosed INT,
    DateCaseClosed DATE
);
"""
cursor.execute(create_table_query)

In [12]:
insert_query = """
INSERT INTO crimes (ReportNumber, DateReported, DateOccurrence, TimeOccurrence, City, CrimeCode,
                    CrimeDescription, VictimAge, VictimGender, WeaponUsed, CrimeDomain, PoliceDeployed,
                    CaseClosed, DateCaseClosed)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Extract only the time part from TimeOccurrence
data['TimeOccurrence'] = data['Time of Occurrence'].apply(lambda x: x.split(' ')[1] if ' ' in x else x)

# Replace 'Not Closed' with None in 'Date Case Closed' column
data['Date Case Closed'] = data['Date Case Closed'].replace('Not Closed', None)

# Select only the relevant columns for the insert query
columns = ['Report Number', 'Date Reported', 'Date of Occurrence', 'TimeOccurrence', 'City', 'Crime Code',
           'Crime Description', 'Victim Age', 'Victim Gender', 'Weapon Used', 'Crime Domain', 'Police Deployed',
           'Case Closed', 'Date Case Closed']
data_records = data[columns].values.tolist()

cursor.executemany(insert_query, data_records)
conn.commit()
cursor.close()
conn.close()

In [14]:
import plotly.express as px

city_crime_counts = data['City'].value_counts().head(10)
px.bar(city_crime_counts, x=city_crime_counts.index, y=city_crime_counts.values,
       title='Top 10 Cities by Crime Frequency', labels={'x': 'City', 'y': 'Crime Count'}).show()

In [15]:
px.box(data, x='Crime Domain', y='Victim Age', color='Crime Domain',
       title='Victim Age Distribution by Crime Domain').show()

In [16]:
weapon_counts = data['Weapon Used'].value_counts().head(10)
px.bar(weapon_counts, x=weapon_counts.index, y=weapon_counts.values,
       title='Top 10 Most Used Weapons', labels={'x': 'Weapon Used', 'y': 'Count'}).show()

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

data['City'] = le.fit_transform(data['City'])

In [26]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['TimeOccurrence', 'Time of Occurrence', 'Case Closed', 'Date Reported', 'Date of Occurrence', 'Date Case Closed'])
y = data['Case Closed']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [30]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 49.63%

Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.50      0.50      4046
           1       0.49      0.49      0.49      3986

    accuracy                           0.50      8032
   macro avg       0.50      0.50      0.50      8032
weighted avg       0.50      0.50      0.50      8032


Confusion Matrix:
 [[2032 2014]
 [2032 1954]]


In [32]:
feature_importances = model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances}).sort_values(by='Importance', ascending=False)

px.bar(importance_df, x='Feature', y='Importance', title='Feature Importance in Predicting Case Closure').show()

In [33]:
data.to_csv('processed_crime_data.csv', index=False)